In [74]:
import numpy
import scipy.special
import scipy.ndimage

In [46]:
class neuralNetwork:
    
    # Инициализация нейронной сети
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        self.lr = learningrate
        
        # Матрицы вес. коэф. wih и who
#         self.wih = (numpy.random.rand(self.hnodes, self.inodes) - 0.5)
#         self.who = (numpy.random.rand(self.onodes, self.hnodes) - 0.5)
        
        # Другой вариант: заполнение весов через нормлаьное распределение
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
        # Использование сигмоиды в качестве функции активации
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass
    
    # Тренировка нейронной сети
    def train(self, inputs_list, targets_list):
        # Преобразовать список входных значений в двумерный массив
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # Рассчитать входящие сигналы для скрытого слоя
        hidden_inputs = numpy.dot(self.wih, inputs)
        # Рассчитать исходящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # Рассчитать входящие сигналы для скрытого слоя
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # Рассчитать исходящие сигналы для скрытого слоя
        final_outputs = self.activation_function(final_inputs)
        
        # Расчет ошибки
        output_errors = targets - final_outputs
        
        # Ошибки скрытого слоя - это ошибки output_errors,
        # Распределение пропорционально весовым коэффициентам связей
        # И рекомбинированные на скрытх узлах
        hidden_errors = numpy.dot(self.who.T, output_errors)
        
        # Обновить весовые коэффициенты между скрытым и выходным слоем
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        # Обновить весовые коэффициенты между входным и скрытым слоем
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass
    
    # Опрос нейронной сети
    def query(self, inputs_list):
        # Преобразовать список входных значений в двумерный массив
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # Расчет входящих сигналов для скрытого слоя
        hidden_inputs = numpy.dot(self.wih, inputs)
        # Расчет исходящих сигналов для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # Расчет входящих сигналов для выходного слоя
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # Расчет исходящих сигналов для выходного слоя
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [86]:
# количество входных, скрытых и выходных узлов 
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# коэффициент обучения
learning_rate = 0.01

# создать экземпляр нейронной сети
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes,learning_rate)

In [87]:
# загрузить в список тренировочный набор данных CSV-файла набора MNIST 
training_data_file = open('mnist_dataset/mnist_train.csv', 'r') 
training_data_list = training_data_file.readlines() 
training_data_file.close()

# загрузить в список тестовый набор данных CSV-файла набора MNIST
test_data_file = open("mnist_dataset/mnist_test.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()

## Тренировка нейронной сети

In [ ]:
# переменная epochs указывает, сколько раз тренировочный набор 
# данных используется для тренировки сети
epochs = 10

for e in range(epochs):

    # перебрать все записи в тренировочном наборе данных 
    for record in training_data_list:
        # получить список значений, используя символы запятой (1,1)
        # в качестве разделителей 
        all_values = record.split(',')
        # масштабировать и сместить входные значения
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # создать целевые выходные значения (все равны 0,01, за исключением
        # желаемого маркерного значения, равного 0,99) 
        targets = numpy.zeros(output_nodes) + 0.01

        # all_values[0] - целевое маркерное значение для данной записи 
        targets[int(all_values[0])] =0.99
        n.train(inputs, targets) 
        
        pass
    
    pass

## Тестирование нейронной сети

In [ ]:
# Журнал оцено работы сети, первоначально пустой
scorecard = []

# Перебрать все записи в тесовом наборе
for record in test_data_list:
    # получить список значений, используя символы запятой (1,1)
    # в качестве разделителей 
    all_values = record.split(',')
    
    # правильный ответ - первое значение
    correct_label = int(all_values[0])
    
    # масштабировать и сместить входные значения
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # опрос сети
    outputs = n.query(inputs)
    # индекс наиб. значения является маркерным значения
    label = numpy.argmax(outputs)
    
    # присоединить оценку ответа сети к концу спсика
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass
    pass



In [ ]:
# рассчитать показатель эффективности доли правильных ответов
scorecard_array = numpy.asarray(scorecard)
print("эффективность = ", scorecard_array.sum() / scorecard_array.size)

In [ ]:
scipy.nd